In [1]:
# パッケージのimport
import os.path as osp
from PIL import Image

import torch.utils.data as data

In [ ]:
def make_datapath_list(rootpath):

    imagepath_template = osp.join(rootpath, 'JPEPGImages', '%s.jpg')
    annopath_template = osp.join(rootpath, 'SegmentionClass', '%s.png')

    train_id_names = osp.join(rootpath + 'ImageSets/Segmentation/train.txt')
    val_id_names = osp.join(rootpath + 'ImageSets/Segmentation/val.txt')

    train_img_list = list()
    train_anno_list = list()

    for line in open(train_id_names):
        file_id = line.strip()
        img_path = (imgpath_tmeplate % file_id)
        annno_path = (annopath_template % file_id)